In [1]:
import toml
import polars as pl
from toolz import get_in
from pathlib import Path

In [2]:
model_path = Path(
    "/n/groups/datta/win/longtogeny/size_norm/models/bottleneck_optimization_00"
)

In [3]:
models = sorted(model_path.glob("**/model.pt"))

In [4]:
df = []
for model in models:
    uuid = model.parent.name
    stage = model.parents[1].name
    config = toml.load(model.with_name("config.toml"))
    arch = config["model"]["lightning"]["arch"]
    use_fft = get_in(["model", arch, "use_fft_branch"], config, False)
    bottleneck = get_in(["model", arch, "bottleneck"], config, 0)
    channels = get_in(["model", "init_channel"], config, 0)
    curriculum_learning = get_in(["model", "lightning", "use_curriculum_learning"], config, False)
    df.append(
        {
            "stage": stage,
            "uuid": uuid,
            "arch": arch,
            "fft_branch": use_fft,
            "bottleneck": bottleneck,
            "channels": channels,
            "curriculum_learning": curriculum_learning,
            "file": str(model),
        }
    )
df = pl.DataFrame(df)

In [5]:
df.write_csv(model_path / "models.csv")